In [20]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization

import warnings
warnings.filterwarnings("ignore")

In [27]:
#----------------
# CONFIGURACIONES
#----------------

IMG_SIZE = (224,224)
BATCH = 16
EPOCHS_HEAD = 15
EPOCHS_FT = 10
DATA_DIR = 'tomato'
CACHE_TRAIN = "temp/train.cache"
CACHE_VAL = "temp/val.cache"

LIMIT_THREADS = True
USE_MIXED_PRECISION = False

In [22]:
gpus = tf.config.list_logical_devices('GPU')

for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

if USE_MIXED_PRECISION and gpus:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')

In [23]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    image_size = IMG_SIZE,
    batch_size = BATCH,
    label_mode = 'int',
    shuffle = True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'val'),
    image_size = IMG_SIZE,
    batch_size = BATCH,
    label_mode = 'int',
    shuffle = False
)


class_names_en = train_ds.class_names


train_ds = train_ds.cache(CACHE_TRAIN)
val_ds = val_ds.cache(CACHE_VAL)

train_ds = train_ds.prefetch(1)
val_ds = val_ds.prefetch(1)

if LIMIT_THREADS:
    opt = tf.data.Options()
    opt.threading.private_threadpool_size = 1
    opt.threading.max_intra_op_parallelism = 1
    #opt.threading.max_inter_op_parallelism = 1
    opt.experimental_optimization.map_parallelization = False

    train_ds = train_ds.with_options(opt)
    val_ds = val_ds.with_options(opt)

Found 10000 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


In [24]:
# ----------------
# CLASES Y MAPEO
# ----------------

CLASS_MAP_ES = {
    'Tomato___Bacterial_spot': 'Mancha bacteriana',
    'Tomato___Early_blight': 'Tizón temprano',
    'Tomato___healthy': 'Sano',
    'Tomato___Late_blight': 'Tizón tardío',
    'Tomato___Leaf_Mold': 'Moho de la hoja',
    'Tomato___Septoria_leaf_spot': 'Mancha foliar por Septoria',
    'Tomato___Spider_mites Two-spotted_spider_mite': 'Ácaro rojo de dos manchas',
    'Tomato___Target_Spot': 'Mancha de tiro',
    'Tomato___Tomato_mosaic_virus': 'Virus del mosaico del tomate',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 'Virus del rizado amarillo de la hoja del tomate',
}

# validar cobertura

missing = [c for c in class_names_en if c not in CLASS_MAP_ES]
extra = [k for k in CLASS_MAP_ES if k not in class_names_en]

assert not missing, f"Faltan traducciones en CLASS_MAP_ES: {missing}"

if extra:
    print(f"Advertencia: hay entradas extra en CLASS_MAP_ES que no están en los datos: {extra}")

# Vector ixs -> ES según el orden real del dataset

IDX2ES = [CLASS_MAP_ES[c] for c in class_names_en]
num_classes = len(class_names_en)
print("Clases (EN):", class_names_en)
print("Clases (ES):", IDX2ES)

Clases (EN): ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']
Clases (ES): ['Mancha bacteriana', 'Tizón temprano', 'Tizón tardío', 'Moho de la hoja', 'Mancha foliar por Septoria', 'Ácaro rojo de dos manchas', 'Mancha de tiro', 'Virus del rizado amarillo de la hoja del tomate', 'Virus del mosaico del tomate', 'Sano']


In [30]:
#--------------------------
# Modelo: MobileNetV3 Small
#--------------------------

base = keras.applications.MobileNetV3Small(
    input_shape = IMG_SIZE + (3,),
    include_top = False,
    weights = 'imagenet',
    alpha = 1.0,
    minimalistic = False
)

base.trainable = False

preprocess = keras.applications.mobilenet_v3.preprocess_input

inputs = keras.Input(shape = IMG_SIZE + (3,))

x = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1)
], name = "auf")(inputs)
x = preprocess(x)
x = base(x, training  = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer = keras.optimizers.Adam(1e-3),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"]
)

cbs_head = [
    keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 1, restore_best_weights= True),
    keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience= 1, min_lr=1e-6)
]

print("\n--- Entrenamiento Fase 1 (cabeza) ----")
model.fit(train_ds, validation_data = val_ds, epochs = EPOCHS_HEAD, callbacks=cbs_head)

# --------------------------------------------------------------------------------------
# Fine-tuning: congelar BN y liberar N capas finales (no BN)
# --------------------------------------------------------------------------------------

for layer in base.layers:
    if not isinstance(layer, BatchNormalization):
        layer.trainable = True

model.compile(
    optimizer = keras.optimizers.Adam(1e-4),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"]
)

cbs_ft = [
    keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 1, restore_best_weights= True),
    keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience= 1)
]

print("\n---- Entrenamiento Fase 2 (fine-tunig) ----")
model.fit(train_ds, validation_data = val_ds, epochs = EPOCHS_FT, callbacks = cbs_ft)

print("\n---- Evaluación en validación ----")
validation_loss, validation_acc = model.evaluate(val_ds)

print(f"Pérdida en validación: {validation_loss:.4f}; Precisión en validación: {validation_acc:.4f}")


--- Entrenamiento Fase 1 (cabeza) ----
Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 92ms/step - accuracy: 0.7660 - loss: 0.6911 - val_accuracy: 0.8210 - val_loss: 0.4908 - learning_rate: 0.0010
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 86ms/step - accuracy: 0.8689 - loss: 0.3905 - val_accuracy: 0.8770 - val_loss: 0.3551 - learning_rate: 0.0010
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 86ms/step - accuracy: 0.8891 - loss: 0.3199 - val_accuracy: 0.8700 - val_loss: 0.3629 - learning_rate: 0.0010

---- Entrenamiento Fase 2 (fine-tunig) ----
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 105s 152ms/step - accuracy: 0.9300 - loss: 0.2095 - val_accuracy: 0.9620 - val_loss: 0.1249 - learning_rate: 1.0000e-04
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 97s 155ms/step - accuracy: 0.9636 - loss: 0.1069 - val_accuracy: 0.9630 - val_loss: 0.1060 - learning_rate: 1.0000e-04
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 93s 149ms/step - accuracy: 0.9778 - loss: 0.0708 - val_accuracy: 0.9450 - val_loss: 0.1386 - l

In [31]:
# ----------------------
# GUARDAR MODELO Y MAPEO
# ----------------------

model.save('mobileNetV3Small.keras')

with open('class_map_es.json', 'w',encoding='utf-8') as f:
    json.dump(
        {"class_names_en": class_names_en, "class_names_es": IDX2ES},
        f,
        ensure_ascii=False,
        indent = 2
    )

In [32]:
# ------------------------
# UTILIDADES DE PREDICCIÓN
# ------------------------

def decode_top1_es(probs):
    idx = int(np.argmax(probs))
    return IDX2ES[idx], float(probs[idx])

def predict_file_es(path):
    img = keras.utils.load_img(path, target_size = IMG_SIZE)
    arr = keras.utils.img_to_array(img)[None, ...]
    arr = keras.applications.mobilenet_v3.preprocess_input(arr)
    probs = model.predict(arr,verbose = 0)[0]
    return decode_top1_es(probs)

In [34]:
img_path = '/mnt/d/Capstone/MobileNetV3/tomato/val/Tomato___Target_Spot/0a2de4c5-d688-4f9d-9107-ace1d281c307___Com.G_TgS_FL 7941.JPG'
etiqueta_es, prob = predict_file_es(img_path)
print(etiqueta_es, prob)

Mancha de tiro 0.9991484880447388
